# 🚀 JetX Model Eğitimi - Google Colab (v7.2)

**6 Model Ensemble Sistemi ile JetX tahmin modellerini eğitin**

---

## 🆕 v7.2 Yeni Özellikler

- 🔧 **İç İçe Klasör Sorunu Düzeltildi**: GitHub'dan direkt açma desteği
- 🤖 **6 Model Tam Desteği**: Progressive NN, CatBoost, AutoGluon, TabNet, Consensus, RL Agent
- 🧠 **AutoGluon AutoML**: 50+ modeli otomatik dener ve en iyisini seçer
- 🎯 **TabNet High-X Specialist**: Attention mechanism ile yüksek çarpanları tespit eder
- 🤖 **RL Agent**: Tüm model çıktılarını birleştirerek en kârlı aksiyonu seçer
- 📁 **Google Drive Entegrasyonu**: Modeller otomatik Drive'a yedeklenir
- 📊 **Kapsamlı Model Karşılaştırma**: Her modelin performans metrikleri
- 💰 **Sanal Kasa Simülasyonu**: ROI ve kazanç oranları
- 📦 **Gelişmiş İndirme Sistemi**: ZIP indirme + Drive backup

---

## 📋 Model Açıklamaları

| Model | Görev | Özellik | Hedef |
|-------|-------|---------|-------|
| **Progressive NN** | Genel amaçlı tahmin | 5 farklı pencere boyutu | 1.5x eşik tahmini |
| **CatBoost** | Gradient boosting | Multi-scale window | 1.5x eşik + kategori |
| **AutoGluon** | AutoML champion | 50+ model ensemble | 1.5x eşik tahmini |
| **TabNet** | Yüksek X uzmanı | Attention mechanism | Multi-class tahmin |
| **Consensus** | Ensemble final | Weighted voting | En yüksek doğruluk |
| **RL Agent** | Meta-model | Policy gradient | En kârlı aksiyon seçimi |

## 🎯 Hedefler

- ✅ 1.5 Altı Doğruluk: **75%+**
- ✅ 1.5 Üstü Doğruluk: **75%+**
- ✅ Para Kaybı Riski: **<20%**
- ✅ ROI: **Pozitif**

## 📦 Adım 1: Hazırlık ve Kurulum

Bu adımda:
- Google Drive bağlantısı yapılır
- Gerekli kütüphaneler yüklenir
- Dizin kontrolü yapılır (GitHub'dan açma desteği)

In [ ]:
import subprocess
import sys
import os
import json
import time
from datetime import datetime

print("="*80)
print("📦 HAZIRLIK - 6 Model Ensemble Sistem v7.2")
print("🆕 RL Agent Entegrasyonu")
print("="*80)

start_time = time.time()

# Google Drive Bağlantısı
print("\n📁 Google Drive bağlanıyor...")
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Drive başarıyla bağlandı!")
    
    drive_path = '/content/drive/MyDrive/JetX_Models_v7.2'
    os.makedirs(drive_path, exist_ok=True)
    print(f"✅ Drive klasörü oluşturuldu: {drive_path}")
except Exception as e:
    print(f"⚠️ Drive bağlantı hatası: {e}")
    drive_path = None

# Kütüphaneleri yükle
print("\n📦 Kütüphaneler yükleniyor...")
!pip install -q tensorflow scikit-learn catboost pandas numpy scipy joblib matplotlib seaborn tqdm PyWavelets nolds autogluon pytorch-tabnet torch

# Proje klonlama - GitHub'dan jetxpredictor'ı indir
print("\n📥 Proje klonlanıyor...")
if not os.path.exists('jetxpredictor'):
    !git clone https://github.com/onndd/jetxpredictor.git
    print("✅ Proje başarıyla klonlandı!")
else:
    print("✅ Proje zaten mevcut")


# Dizin kontrolü - GitHub'dan direkt açma desteği
print("\n📂 Dizin kontrolü yapılıyor...")
print(f"Mevcut dizin: {os.getcwd()}")

# Proje kök dizinini bul
if os.path.exists('jetx_data.db'):
    print("✅ Proje kök dizinindeyiz!")
elif os.path.exists('../jetx_data.db'):
    print("📁 Bir üst dizine çıkılıyor...")
    os.chdir('..')
    print(f"✅ Proje kök dizinine geçildi: {os.getcwd()}")
elif os.path.exists('jetxpredictor/jetx_data.db'):
    print("📁 jetxpredictor klasörüne giriliyor...")
    os.chdir('jetxpredictor')
    print(f"✅ Proje kök dizinine geçildi: {os.getcwd()}")
else:
    print("⚠️ UYARI: jetx_data.db bulunamadı!")
    print(f"📂 Mevcut dizin içeriği: {os.listdir('.')}")
    
# Models klasörünü kontrol et ve oluştur
if not os.path.exists('models'):
    os.makedirs('models')
    print("✅ models/ klasörü oluşturuldu")
else:
    print("✅ models/ klasörü mevcut")

prep_time = time.time() - start_time
print(f"\n✅ Hazırlık tamamlandı! ({prep_time/60:.1f} dakika)")
print(f"📂 Çalışma dizini: {os.getcwd()}")
print("="*80)

## 📊 Adım 2: Veri Yükleme ve Hazırlık

Veritabanından veri yüklenip train/val/test olarak bölünür.

In [ ]:
import numpy as np
import pandas as pd
import sqlite3
from sklearn.preprocessing import StandardScaler
import sys
sys.path.append(os.getcwd())

from category_definitions import CategoryDefinitions, FeatureEngineering
from utils.multi_scale_window import split_data_preserving_order

print("="*80)
print("📊 VERİ YÜKLEME")
print("="*80)

conn = sqlite3.connect('jetx_data.db')
data = pd.read_sql_query("SELECT value FROM jetx_results ORDER BY id", conn)
conn.close()

all_values = data['value'].values
print(f"✅ {len(all_values):,} veri yüklendi")

# Time-series split
train_data, val_data, test_data = split_data_preserving_order(
    all_values, train_ratio=0.70, val_ratio=0.15
)

print(f"✅ Train: {len(train_data):,}")
print(f"✅ Val:   {len(val_data):,}")
print(f"✅ Test:  {len(test_data):,}")
print("="*80)

## 🧠 Adım 3: Progressive NN Training

Multi-scale window ensemble ile Progressive Neural Network eğitimi.

In [ ]:
print("="*80)
print("🧠 PROGRESSIVE NN TRAINING")
print("="*80)

!python notebooks/jetx_PROGRESSIVE_TRAINING_MULTISCALE.py

# Sonuçları yükle
with open('models/progressive_multiscale/model_info.json', 'r') as f:
    progressive_results = json.load(f)

print("\n✅ Progressive NN Tamamlandı!")
print(f"📊 MAE: {progressive_results['ensemble_metrics']['mae']:.4f}")

## 🚀 Adım 4: CatBoost Training

Gradient boosting uzmanı CatBoost modeli eğitimi.

In [ ]:
print("="*80)
print("🚀 CATBOOST TRAINING")
print("="*80)

!python notebooks/jetx_CATBOOST_TRAINING_MULTISCALE.py

with open('models/catboost_multiscale/model_info.json', 'r') as f:
    catboost_results = json.load(f)

print("\n✅ CatBoost Tamamlandı!")

## 🤖 Adım 5: AutoGluon AutoML Training

50+ modeli otomatik deneyen AutoGluon AutoML eğitimi.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from utils.autogluon_predictor import AutoGluonPredictor

print("="*80)
print("🤖 AUTOGLUON AUTOML TRAINING")
print("="*80)

# Feature extraction
window_size = 100
X_features = []
y_labels = []

for i in range(window_size, len(train_data) - 1):
    hist = train_data[:i].tolist()
    target = train_data[i]
    feats = FeatureEngineering.extract_all_features(hist)
    X_features.append(feats)
    y_labels.append(1 if target >= 1.5 else 0)

X_train_ag = pd.DataFrame(X_features)
y_train_ag = pd.Series(y_labels, name='above_threshold')

print(f"✅ {len(X_train_ag):,} örnek hazırlandı")

# AutoGluon predictor
ag_predictor = AutoGluonPredictor(
    model_path='models/autogluon_model',
    threshold=1.5
)

# Train
ag_results = ag_predictor.train(
    X_train=X_train_ag,
    y_train=y_train_ag,
    time_limit=3600,
    presets='best_quality',
    eval_metric='roc_auc'
)

print("\n✅ AutoGluon Tamamlandı!")
print(f"🏆 En İyi Model: {ag_results['best_model']}")
print(f"📊 Score: {ag_results['best_score']:.4f}")

# Save info
autogluon_info = {
    'model': 'AutoGluon_AutoML',
    'version': '1.0',
    'date': datetime.now().strftime('%Y-%m-%d'),
    'best_model': ag_results['best_model'],
    'best_score': float(ag_results['best_score'])
}

os.makedirs('models/autogluon_model', exist_ok=True)
with open('models/autogluon_model/model_info.json', 'w') as f:
    json.dump(autogluon_info, f, indent=2)

## 🎯 Adım 6: TabNet High-X Specialist Training

Attention mechanism ile yüksek çarpanları tespit eden TabNet eğitimi.

In [ ]:
from utils.tabnet_predictor import TabNetHighXPredictor

print("="*80)
print("🎯 TABNET HIGH-X SPECIALIST TRAINING")
print("="*80)

# Feature extraction
window_size = 100
X_features_tn = []
y_categories = []

for i in range(window_size, len(train_data) - 1):
    hist = train_data[:i].tolist()
    target = train_data[i]
    feats = FeatureEngineering.extract_all_features(hist)
    X_features_tn.append(list(feats.values()))
    category = TabNetHighXPredictor.categorize_value(target)
    y_categories.append(category)

X_train_tn = np.array(X_features_tn)
y_train_tn = np.array(y_categories)

scaler_tn = StandardScaler()
X_train_tn = scaler_tn.fit_transform(X_train_tn)

print(f"✅ {len(X_train_tn):,} örnek hazırlandı")

# Validation set
X_val_tn = []
y_val_tn = []

for i in range(window_size, len(val_data) - 1):
    hist = val_data[:i].tolist()
    target = val_data[i]
    feats = FeatureEngineering.extract_all_features(hist)
    X_val_tn.append(list(feats.values()))
    y_val_tn.append(TabNetHighXPredictor.categorize_value(target))

X_val_tn = scaler_tn.transform(np.array(X_val_tn))
y_val_tn = np.array(y_val_tn)

# TabNet predictor
tabnet_predictor = TabNetHighXPredictor(
    model_path='models/tabnet_high_x.pkl',
    scaler_path='models/tabnet_scaler.pkl'
)

# Train
tn_results = tabnet_predictor.train(
    X_train=X_train_tn,
    y_train=y_train_tn,
    X_val=X_val_tn,
    y_val=y_val_tn,
    max_epochs=200,
    patience=20,
    batch_size=256
)

print("\n✅ TabNet Tamamlandı!")
print(f"🏆 Best Epoch: {tn_results['best_epoch']}")

# Save
tabnet_predictor.save_model()
tabnet_predictor.save_scaler(scaler_tn)

tabnet_info = {
    'model': 'TabNet_HighX_Specialist',
    'version': '1.0',
    'date': datetime.now().strftime('%Y-%m-%d'),
    'best_epoch': int(tn_results['best_epoch']),
    'best_cost': float(tn_results['best_cost'])
}

with open('models/tabnet_info.json', 'w') as f:
    json.dump(tabnet_info, f, indent=2)

## 🤖 Adım 7: RL Agent Training

Reinforcement Learning Agent eğitimi. Tüm model çıktılarını birleştirerek en kârlı aksiyonu seçer.


In [ ]:
print("="*80)
print("🤖 RL AGENT TRAINING")
print("="*80)

!python notebooks/train_rl_agent.py

# Sonuçları yükle
try:
    with open('models/rl_agent_info.json', 'r') as f:
        rl_agent_info = json.load(f)
    print("\n✅ RL Agent Tamamlandı!")
    print(f"📊 Accuracy: {rl_agent_info.get('accuracy', 0):.2%}")
    print(f"📊 Action Distribution: {rl_agent_info.get('action_distribution', [0, 0, 0, 0])}")
except Exception as e:
    print(f"\n⚠️ RL Agent info yüklenemedi: {e}")
    rl_agent_info = None


## 💾 Adım 8: Sonuçları Kaydetme ve İndirme

Tüm modeller ZIP'lenir ve indirilir. Google Drive'a yedekleme yapılır.

In [ ]:
import shutil

print("="*80)
print("💾 SONUÇLAR KAYDEDİLİYOR")
print("="*80)

# JSON sonuçları
final_results = {
    'metadata': {
        'version': '7.2',
        'date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    },
    'models': {
        'progressive_nn': progressive_results if 'progressive_results' in locals() else None,
        'catboost': catboost_results if 'catboost_results' in locals() else None,
        'autogluon': autogluon_info if 'autogluon_info' in locals() else None,
        'tabnet': tabnet_info if 'tabnet_info' in locals() else None,
        'rl_agent': rl_agent_info if 'rl_agent_info' in locals() else None
    }
}

with open('models/all_models_results_v7.json', 'w') as f:
    json.dump(final_results, f, indent=2)

print("✅ JSON sonuçları kaydedildi")

# Lokal Streamlit uyumluluğu için dosya isimlendirme
print("\n📝 Lokal kullanım için dosyalar hazırlanıyor...")

# Progressive NN modelini ana model olarak kopyala
if os.path.exists('models/progressive_multiscale/ensemble_model.h5'):
    shutil.copy('models/progressive_multiscale/ensemble_model.h5', 'models/jetx_model.h5')
    print("✅ jetx_model.h5 oluşturuldu")

if os.path.exists('models/progressive_multiscale/scaler.pkl'):
    shutil.copy('models/progressive_multiscale/scaler.pkl', 'models/scaler.pkl')
    print("✅ scaler.pkl oluşturuldu")

# ZIP oluştur
zip_filename = f'jetx_6models_v7_{datetime.now().strftime("%Y%m%d_%H%M")}'
shutil.make_archive(zip_filename, 'zip', 'models')

print(f"\n✅ ZIP oluşturuldu: {zip_filename}.zip")

# Drive'a yedekle
if drive_path:
    try:
        drive_zip = f"{drive_path}/{zip_filename}.zip"
        shutil.copy(f"{zip_filename}.zip", drive_zip)
        print(f"✅ Drive'a yedeklendi: {drive_zip}")
    except Exception as e:
        print(f"⚠️ Drive yedekleme hatası: {e}")

# İndirme
try:
    from google.colab import files
    files.download(f'{zip_filename}.zip')
    print(f"\n✅ Dosya indiriliyor...")
    print("\n📌 İNDİRME TALİMATLARI:")
    print("="*80)
    print("1. ZIP dosyasını masaüstünde açın")
    print("2. İçindeki tüm dosyaları jetxpredictor/models/ klasörüne kopyalayın")
    print("3. Terminal'de: streamlit run app.py")
    print("4. \"✅ Model yüklendi ve hazır!\" mesajını görmelisiniz")
    print("="*80)
except:
    print(f"\n📁 Dosya konumu: {os.getcwd()}/{zip_filename}.zip")
    print("Manuel indirme: Dosyalar > Sağ tık > İndir")

print("\n" + "="*80)
print("✅ TÜM İŞLEMLER TAMAMLANDI!")
print("="*80)

## 🎉 Adım 9: Final Özet

Eğitim süreci tamamlandı. Sonuçları görüntüleyin.

In [ ]:
print("="*80)
print("🎉 JetX 6 MODEL ENSEMBLE SİSTEMİ - FİNAL ÖZET")
print("="*80)

print("\n📊 EĞİTİLEN MODELLER:")
model_count = 0

if 'progressive_results' in locals():
    model_count += 1
    print(f"   1️⃣ Progressive NN (Multi-Scale) ✅")
    print(f"      - MAE: {progressive_results['ensemble_metrics']['mae']:.4f}")

if 'catboost_results' in locals():
    model_count += 1
    print(f"\n   2️⃣ CatBoost (Multi-Scale) ✅")

if 'autogluon_info' in locals():
    model_count += 1
    print(f"\n   3️⃣ AutoGluon AutoML ✅")
    print(f"      - Best Model: {autogluon_info['best_model']}")

if 'tabnet_info' in locals():
    model_count += 1
    print(f"\n   4️⃣ TabNet High-X Specialist ✅")

if 'rl_agent_info' in locals():
    model_count += 1
    print(f"\n   5️⃣ RL Agent ✅")
    print(f"      - Accuracy: {rl_agent_info.get('accuracy', 0):.2%}")

print(f"\n📈 TOPLAM: {model_count}/6 model başarıyla eğitildi")

print("\n📁 KAYDEDILEN DOSYALAR:")
print(f"   ✅ models/jetx_model.h5 (Ana Progressive NN)")
print(f"   ✅ models/scaler.pkl (Feature scaler)")
print(f"   ✅ models/rl_agent_model.h5 (RL Agent)")
print(f"   ✅ models/rl_agent_model_scaler.pkl (RL Agent Scaler)")
print(f"   ✅ models/all_models_results_v7.json")
print(f"   ✅ {zip_filename}.zip")
if drive_path:
    print(f"   ✅ {drive_path}/{zip_filename}.zip (Drive backup)")

print("\n" + "="*80)
print("✨ BAŞARIYLA TAMAMLANDI! ✨")
print("="*80)
print(f"Bitiş: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)